In [1]:
# Import library
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import os.path
import re
import logging
import sys
import requests

In [2]:
# Vào trang chủ danh mục sách (lọc theo các sản phẩm bán chạy để lấy các sản phẩm có lượt đánh giá)
driver = webdriver.Chrome() 
driver.get('https://tiki.vn/search?q=s%C3%A1ch&ref=searchBar&sort=top_seller&page=1')

In [3]:
def get_link_by_text(text):
    """Find link in the page with given text"""
    element = driver.find_element_by_link_text(text.strip())
    return element.get_attribute("href")

def get_list_by_class_name(class_name="main_image "):
    """Get list of text in all element by class_name"""
    element_list = []
    try:
        all_elements = driver.find_elements_by_class_name(class_name)
        element_list = [x.text for x in all_elements if len(x.text) > 0]
    except (NoSuchElementException, WebDriverException) as e:
        print(e)
    return element_list

In [4]:
# Lấy 40 link trang trong mục sản phẩm bán chạy
page_num = driver.find_elements_by_xpath("//a[@data-view-id='product_list_pagination_item']")
pages_link = [x.get_attribute('href') for x in page_num]
time.sleep(3)
list_links_page =[]
n = 1
while n <= 40:
    list_links_page.append(pages_link[0].replace("page=1", "page="+str(n)))
    n = n + 1

In [5]:
# Lấy link tất cả các sản phẩm trong 40 trang đã lấy ở trên
product_urls = {}
for page in list_links_page:
    driver.get(page)
    time.sleep(3)
    category_links = {x: get_link_by_text(x)
                      for x in get_list_by_class_name('product-item')}
    product_urls = dict(product_urls, **category_links)
    time.sleep(3)


list_link_new = []
for category, url in product_urls.items():
        list_link_new.append(url)
print(len(list_link_new))

768


In [6]:
# Lấy review cho các link sản phẩm đã thu được ở trên và chuyển vào DataFrame
def review_text_by_link(url, rating_url):
    d = {"customerID": [],"title":[], "content": [], "rating": []}
    r = re.search(r"(\d+)\.", url)
    product_id = r[1]
    offset = 0
    params = {
        'limit':900,
        'include': 'comments',
        'product_id': product_id,
        'sort': 'core|decs,id|decs,star|all'}
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    data = requests.get(rating_url, params=params, headers=headers)
    if data.status_code != 200:
        return "Error with status: " + str(data.status_code)
    data = data.json()
    time.sleep(2)
    for n, rating in enumerate(data["data"], 1):
        d["customerID"].append(rating["customer_id"])
        d["rating"].append(rating["rating"])
        d["title"].append(rating["title"])
        d["content"].append(rating["content"]) 
    df = pd.DataFrame(d)
    return df

In [7]:
rating_url = "https://tiki.vn/api/v2/reviews"
df = pd.DataFrame()
for url in list_link_new[:100]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [8]:
for url in list_link_new[100:200]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [9]:
for url in list_link_new[200:300]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [10]:
for url in list_link_new[400:500]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [11]:
for url in list_link_new[500:700]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [12]:
for url in list_link_new[700:900]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [13]:
for url in list_link_new[900:1100]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [14]:
for url in list_link_new[1100:1300]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [15]:
for url in list_link_new[1300:1500]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [16]:
for url in list_link_new[1500:1700]:
    a = review_text_by_link(url, rating_url)
    df = pd.concat([df, a])

In [17]:
# for url in list_link_new[1700:1943]:
#     a = review_text_by_link(url, rating_url)
#     df = pd.concat([df, a])

In [18]:
print(df.shape)

(27782, 4)


In [19]:
data = df

In [20]:
# Thêm cột 'review_text' là dữ liệu của 2 cột 'title' và 'content'
df['review_text'] = df['content'] 
df.drop(['title', 'content'], axis=1, inplace=True)

In [21]:
# Loại bỏ NaN và Null trong cột 'review_text'
df.replace("", np.nan, inplace=True)
df.dropna(subset=['review_text'], inplace=True)
# Xoá các dòng trùng lặp
df.drop_duplicates(inplace=True, ignore_index=False)
df.reset_index(drop=True, inplace=True)

In [22]:
# Số dòng, số cột
df.shape

(10429, 3)

In [23]:
df['ratings'] = df['rating']
df = df.drop(columns = ['customerID', 'rating'])
df = df.rename(columns = {'ratings' : 'rating', 'review_text': 'text'})

In [24]:
df = df.loc[df.rating < 4]

In [25]:
df = df.reset_index().drop(columns = ['index'])

In [26]:
df1 = df.copy()
for i in range(df1.shape[0]):
    if 'giao hàng' in df1.text[i] or 'dịch vụ' in df1.text[i] or 'vận chuyển' in df1.text[i] or 'sách cũ' in df1.text[i]\
    or 'đóng gói' in df1.text[i] or '.. ..' in df1.text[i] or 'Giao hàng' in df1.text[i] or 'bookcare' in df1.text[i]\
    or 'Bookcare' in df1.text[i] or 'bookmart' in df1.text[i] or 'ship' in df1.text[i] or 'shipper' in df1.text[i]\
    or 'Shipper' in df1.text[i] or 'giao' in df1.text[i] or 'Giao' in df1.text[i] or 'bookmar' in df1.text[i]\
    or 'bìa' in df1.text[i] or 'Bìa' in df1.text[i] or 'gửi' in df1.text[i] or 'Bookmark' in df1.text[i]\
    or 'Tiki' in df1.text[i] or 'tiki' in df1.text[i] or 'shop' in df1.text[i] or 'Shop' in df1.text[i]\
    or 'Đóng gói' in df1.text[i] or 'cong' in df1.text[i] or 'gãy' in df1.text[i] or 'dơ' in df1.text[i] \
    or 'đơn hàng' in df1.text[i] or 'bẩn' in df1.text[i]:
        df1 = df1.drop(i)

In [30]:
# # Lưu Dataframe vào file csv
df1 = df1.reset_index().drop(columns = ['index'])
df1.to_csv('crawling_tiki.csv')

In [34]:
df1.to_csv('crawling_tiki.csv')

In [ ]:
# df1.to_csv('Tiki_1.csv', encoding='utf-8-sig')